In [17]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [18]:
import pandas as pd
import os

# Load Arguments Dataset
data_folder = './data/'
arguments_file = 'arguments-training.tsv'
labels_file = 'labels-training.tsv'
arguments_train_df = pd.read_csv(os.path.join(data_folder, arguments_file), encoding='utf-8', sep='\t', header=0)
labels_train_df = pd.read_csv(os.path.join(data_folder, labels_file), encoding='utf-8', sep='\t', header=0)

print(arguments_train_df)
print(labels_train_df)

     Argument ID                      Conclusion       Stance  \
0         A01001  Entrapment should be legalized  in favor of   
1         A01002     We should ban human cloning  in favor of   
2         A01003      We should abandon marriage      against   
3         A01004       We should ban naturopathy      against   
4         A01005         We should ban fast food  in favor of   
...          ...                             ...          ...   
5215      D27096    Nepotism exists in Bollywood      against   
5216      D27097    Nepotism exists in Bollywood  in favor of   
5217      D27098         India is safe for women  in favor of   
5218      D27099         India is safe for women  in favor of   
5219      D27100         India is safe for women      against   

                                                Premise  
0     if entrapment can serve to more easily capture...  
1     we should ban human cloning as it will only ca...  
2     marriage is the ultimate commitment to 

In [20]:
from datasets import Dataset

# Combine the columsn in arguments to be a single field to give to bert

# Inputs: 
# an argument df from the source data (ArgumentId, Conclusion, Stance, Premise). 
# Labels df from file. 
# Name of label that will be trained on.

# Returns: df with a single column of arguments that is Conclusion: Conclusion, Stance: stance, Premise: Premise 
# along with the labels
def setup_train_df(arguments_df, labels_df, target_label):
    arguments_df['text'] = 'Conclusion: ' + arguments_df['Conclusion'] + ', Stance: ' + arguments_df['Stance'] + ', Premise: ' + arguments_df['Premise']
    resp = arguments_df.filter(['text'], axis=1)
    resp['label'] = labels_df[target_label]
    return resp

# This is where the specific value label is selected.
target_label = 'Achievement'
train = setup_train_df(arguments_train_df, labels_train_df, target_label)

print(dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 5220
})


In [ ]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

# Do some huggingface/transformers setup
dataset = Dataset.from_pandas(train)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
tokenized_arguments = dataset.map(preprocess_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Do training here
# This is based on this guide: https://huggingface.co/docs/transformers/tasks/sequence_classification
# TODO: Split train/test data. Optimizer. Fine tuning. Get the training to run. Evaluation.
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_arguments["train"],
    eval_dataset=tokenized_arguments["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()